In [11]:
%cd ..

# imports:
import pyOSC3

# todo: hide/combine/simplify these imports
from domblar.sc3.client import SC3Client
from domblar.sc3.instruments import setup_instruments, assign_instrument
from domblar.players import play, play_voices
from domblar.transformations import iterate_voice_leadings, shift, transpose
from domblar.rhythm import get_grouped_rhythm, rhythm_to_notes
from domblar.edo import build_mos_from_edo, intervals_in_edo

/Users/gexahedron


In [7]:
# setup:
# 1. open scds/vst_tracker.scd file in SuperCollider
# and run first cell
# and second cell
# and third cell

# 2. run this cell
client = SC3Client()
instruments = setup_instruments()

In [3]:
# play:

def assign_instruments():
    # assign_instrument(0, 'tub_bells', instruments, client)
    assign_instrument(0, 'brass/toto_africa', instruments, client)
assign_instruments()

edo = 37
# fixme: remove this scale, looks redundant
scale = list(range(edo))

scales = {}
scales['orgone7'] = build_mos_from_edo(7, gen=10, edo=edo, down=0, shift=0) + [edo]

cur_scale = scales['orgone7']
print(cur_scale)
print('steps -', intervals_in_edo(cur_scale, edo))
play(cur_scale, scale, edo, client, synth_idx=[0], dur=0.3)


[0, 3, 10, 13, 20, 23, 30, 37]
steps - [3, 7, 3, 7, 3, 7, 7]


In [4]:
chords = iterate_voice_leadings(
    [0, 2, 7, 11],
    [-2, 10, -2, -2],
    [0, 2, 3, 1],
    7)
print(chords)

def assign_instruments():
    for i in range(4):
        # assign_instrument(i, 'tub_bells', instruments, client)
        assign_instrument(i, 'brass/toto_africa', instruments, client)
assign_instruments()

edo = 12
scale = list(range(edo))
play(chords, scale, edo, client, synth_idx=[0, 1, 2, 3], dur=0.75)


[[0, 2, 7, 11], [-2, 5, 9, 12], [-4, 7, 10, 15], [-6, 8, 13, 17], [-8, 11, 15, 18], [-10, 13, 16, 21], [-12, 14, 19, 23]]


In [10]:
# Steve Reich - "Clapping Music"

from copy import copy
import random

def assign_instruments():
    assign_instrument(0, 'timpani', instruments, client)
    assign_instrument(1, 'timpani2', instruments, client)
assign_instruments()

edo = 2
scale = list(range(edo))

rhythm1 = get_grouped_rhythm([3, 2, 1, 2])
rhythm2 = copy(rhythm1)
voice1 = []
voice2 = []
for i in range(len(rhythm1) + 1):
    rep_count = random.randint(4, 8)
    voice1.extend(rhythm1 * rep_count)
    voice2.extend(rhythm2 * rep_count)
    rhythm2 = shift(rhythm2, 1)
voice1 = rhythm_to_notes(voice1, 0)
voice2 = rhythm_to_notes(voice2, 0)
chords = list(zip(voice1, voice2))
play(chords, scale, edo, client, synth_idx=[0, 1], dur=0.16)

KeyboardInterrupt: 

In [58]:
# melody example
# Stephane Picq, Dune - Water
# what i like about this is
# - simplicity / repetitiveness
# - nice modulations
# values below are MIDI values

def arpeggio_dune_water(notes, reps=2):
    assert (len(notes) == 4)
    notes.sort()
    v1 = [notes[3], notes[2], notes[1], notes[2]]
    v2 = [notes[3], notes[2], notes[0], notes[2]]
    return (v1 + v2) * reps

# c3 = 48; melody starts on d#3, 51, it's d# lydian actually
# vibraphone
part1 = arpeggio_dune_water([43, 45, 46, 51])
part2 = arpeggio_dune_water([43, 45, 46, 50])
part3 = arpeggio_dune_water([43, 45, 47, 52])
part4 = arpeggio_dune_water([47, 49, 50, 54])
# part1, part2, part3, part4 are repeated again

# 1: 51 d# lydian
# 2: 50 d aeolian
# 3: 52 e aeolian
# 4: 47 b aeolian

part5 = arpeggio_dune_water([50, 52, 53, 57])
part6 = arpeggio_dune_water([53, 55, 56, 60])
part7 = arpeggio_dune_water([45, 47, 48, 52])
# part8 is a final part and is more tricky:
part8 = arpeggio_dune_water([44, 47, 48, 51], reps=1) + arpeggio_dune_water([44, 46, 48, 51], reps=1)

# 5: 50 d aeolian (again, but shifted)
# 6: 53 f aeolian
# 7: 45 a aeolian
# 8: no single (Pressing scale) collection, because 2 consecutive semitones
# 8.1: e harm maj, c harm min, oct(0,2), hex(0,3)
# 8.2: 44 g# ionian

def distribute(voice):
    voice1 = []
    voice2 = []
    for i, n in enumerate(voice):
        n1 = '.'
        n2 = n
        if i % 2:
            n1, n2 = n2, n1
        voice1.append(n1)
        voice2.append(n2)
    return list(zip(voice1, voice2))

def assign_instruments():
    for i in range(2):
        assign_instrument(i, 'vibraphone/dune_water', instruments, client)
assign_instruments()

edo = 12
scale = list(range(edo))

melody = part1+part2+part3+part4+part1+part2+part3+part4+part5+part6+part7+part8
melody = transpose(melody, -60)  # because i used MIDI values above
melody = distribute(melody)
melody = melody[:-1] + [['.', '.']]
play(melody, scale, edo, client, synth_idx=[0, 1], dur=0.25)

KeyboardInterrupt: 

In [13]:
# cleanup
# TODO